In [1]:
import numpy as np
import time 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
from jupyterthemes import jtplot

In [3]:
dat_lin = np.arange(0,max_depth[0] + 1,1)

In [19]:
def He3_temps(MAT, T_AMP, dat_lin):
    %run C:\Users\dennis\STEIN_codes\helium\He_3_after_Tremblay.ipynb
    %run C:\Users\dennis\STEIN_codes\temperature\new_crank_nicolson_fixed.ipynb
        
    T0 = Temp_fun(MAT, T_AMP, dat_lin, (1/thermnt))
    EDT_import = EDT_calculation(MAT, T_AMP, T0)
    EDT_trnsfrm = interp1d(x = EDT_import[5], y = EDT_import[3], 
                        kind = 'cubic',
                        fill_value = "extrapolate")
    EDT_z = EDT_trnsfrm(x = dat_lin)
    
    return EDT_z

In [25]:
def He3_fxn(grain_conc, EDT_z):
        
    He3P0 = SLHL_He3 * np.exp(-mu * dat_lin)
    grain_conc = [MT_He_3(He3P0[i], grain_conc[i], x, 1, dx, EDT_z[i], nx) for i in range(0,len(dat_lin))]

    return grain_conc

In [1]:
def He3_loop_fxn(total_time, time_ER_shift):
        %run C:\Users\dennis\STEIN_codes\helium\He_3_after_Tremblay.ipynb
        %run C:\Users\dennis\STEIN_codes\temperature\new_crank_nicolson_fixed.ipynb
        
        conc3He = [[0]*nx] * len(dat_lin)
        He_surf = np.empty((total_time,))
        
        start_time = time.time()
        
        EDT_z = He3_temps(MAT, T_AMP, dat_lin)
        
        for i in range(total_time):
            conc3He = He3_fxn(conc3He, EDT_z)
            He_surf[i] = np.average([np.average(conc3He[i], weights = shell_mass) for i in range(6)])
            
        if save_output == True:
            np.savetxt('NE_He3_ER0.0_surfmat.csv',
                       He_surf,
                       delimiter = ',')
                
        return conc3He, He_surf